In [5]:
import requests
import socket
import ipinfo
import ssl
from cryptography import x509
from cryptography.hazmat.backends import default_backend

from urllib import parse, robotparser, request
import re
import json


ipinfo_api_key = '3f709cf966c756'


#### Gets the IP adress of the given website

In [25]:
def ip_address(website):
    if "://" in website:
        website = website.split("://")[1]

    parts = website.split("/")
    domain = parts[0]
    try:
        ip_address = socket.gethostbyname(domain)
        return (domain, ip_address)
    except socket.gaierror:
        return (domain, "Invalid domain or unable to resolve IP address")


In [24]:
ip_address("http://dsu.edu")

('dsu.edu', '23.99.192.132')

#### Gets the IP information of the given IP address

In [11]:

def ip_info(ip_address):
    handler = ipinfo.getHandler(ipinfo_api_key)

    details = handler.getDetails(ip_address)
    return json.dumps(details.all)

In [12]:
ip_info('23.99.192.132')

'{"ip": "23.99.192.132", "city": "Des Moines", "region": "Iowa", "country": "US", "loc": "41.6005,-93.6091", "org": "AS8075 Microsoft Corporation", "postal": "50307", "timezone": "America/Chicago", "country_name": "United States", "isEU": false, "country_flag": {"emoji": "\\ud83c\\uddfa\\ud83c\\uddf8", "unicode": "U+1F1FA U+1F1F8"}, "country_currency": {"code": "USD", "symbol": "$"}, "continent": {"code": "NA", "name": "North America"}, "latitude": "41.6005", "longitude": "-93.6091"}'

#### Gets the SSL certificate of the given IP adress

In [91]:
def get_ssl(hostname, port=443):
    context = ssl.create_default_context()
    conn = context.wrap_socket(socket.socket(socket.AF_INET, socket.SOCK_STREAM), server_hostname=hostname)
    conn.connect((hostname, port))

    certs = conn.getpeercert(True)
    certificate = x509.load_der_x509_certificate(certs, default_backend())
    subject = next((attr.value for attr in certificate.subject if attr.oid == x509.NameOID.COMMON_NAME), None)
    issuer = next((attr.value for attr in certificate.issuer if attr.oid == x509.NameOID.ORGANIZATION_NAME ), None)
    certificate_info = {
        "subject": subject,
        "issuer": issuer,
        "serial_number": certificate.serial_number,
        "not_valid_before": certificate.not_valid_before.isoformat(),
        "not_valid_after": certificate.not_valid_after.isoformat()
    }

    return json.dumps(certificate_info)
def get_records(domain):
    results = {}
    record_types = ['NONE', 'A', 'NS', 'MD', 'MF', 'CNAME', 'SOA', 'MB', 'MG', 'MR', 'NULL', 'WKS', 'PTR', 'HINFO', 'MINFO', 'MX', 'TXT', 'RP', 'AFSDB', 'X25', 'ISDN', 'RT', 'NSAP', 'NSAP-PTR', 'SIG', 'KEY', 'PX', 'GPOS', 'AAAA', 'LOC', 'NXT', 'SRV', 'NAPTR', 'KX', 'CERT', 'A6', 'DNAME', 'OPT', 'APL', 'DS', 'SSHFP', 'IPSECKEY', 'RRSIG', 'NSEC', 'DNSKEY', 'DHCID', 'NSEC3', 'NSEC3PARAM', 'TLSA', 'HIP', 'CDS', 'CDNSKEY', 'CSYNC', 'SPF', 'UNSPEC', 'EUI48', 'EUI64', 'TKEY', 'TSIG', 'IXFR', 'AXFR', 'MAILB', 'MAILA', 'ANY', 'URI', 'CAA', 'TA', 'DLV']
    
    for record_type in record_types:
        try:
            answers = dns.resolver.resolve(domain, record_type)
            results[record_type] = [rdata.to_text() for rdata in answers]
        except dns.resolver.NoAnswer:
            pass
        except dns.resolver.NXDOMAIN:
            pass
        except Exception as e:
            continue
    
    return json.dumps(results)
def ip_info(ip_address):
    handler = ipinfo.getHandler(ipinfo_api_key)

    details = handler.getDetails(ip_address)

 
    return json.dumps(details.all)
def get_sitemaps(website):
    robotstxturl = parse.urljoin(website, "robots.txt")
    try:
        rp = robotparser.RobotFileParser()
        rp.set_url(robotstxturl)
        rp.read()
        sitemaps = rp.site_maps()
    except robotparser.RobotFileParserError as e:
        print(f"error: {e}")
    except Exception as e:
        print(f"Error: {e}")
        
    return sitemaps

def sitemap_parser(sitemap):
    r = request.urlopen(sitemap)
    xml = r.read().decode('utf8')
    elements = re.findall(r'<loc>(.*?)<\/loc>', xml, re.DOTALL)

    urls = []

    for element in elements:
        if element.endswith('.xml'):
            urls.extend(sitemap_parser(element))  # Recursively call sitemap_parser
        else:
            urls.append(element)

    return urls

def site_maps(url):
    sitemaps = get_sitemaps(url)
    
    all_urls = []
    
    for sitemap in sitemaps:
        all_urls.extend(sitemap_parser(sitemap))

    urls_json = json.dumps(all_urls)
    
    return(urls_json)

#### Reverse DNS

In [30]:
import dns.resolver





#### Sitemap Parser

#### WHOIS Domain

In [43]:
import whois

res = whois.whois('https://dsu.edu')


In [44]:
(res)

{'domain_name': 'DSU.EDU',
 'registrar': None,
 'whois_server': None,
 'referral_url': None,
 'updated_date': None,
 'creation_date': None,
 'expiration_date': None,
 'name_servers': None,
 'status': None,
 'emails': 'abuse@dsu.edu',
 'dnssec': None,
 'name': None,
 'org': None,
 'address': None,
 'city': None,
 'state': None,
 'registrant_postal_code': None,
 'country': None}

In [45]:
from ipwhois import IPWhois

ip = '23.99.192.132'
obj = IPWhois(ip)
res = obj.lookup_rdap()

In [46]:
res

{'nir': None,
 'asn_registry': 'arin',
 'asn': '8075',
 'asn_cidr': '23.96.0.0/14',
 'asn_country_code': 'US',
 'asn_date': '2013-06-18',
 'asn_description': 'MICROSOFT-CORP-MSN-AS-BLOCK, US',
 'query': '23.99.192.132',
 'network': {'handle': 'NET-23-96-0-0-1',
  'status': ['active'],
  'remarks': None,
  'notices': [{'title': 'Terms of Service',
    'description': 'By using the ARIN RDAP/Whois service, you are agreeing to the RDAP/Whois Terms of Use',
    'links': ['https://www.arin.net/resources/registry/whois/tou/']},
   {'title': 'Whois Inaccuracy Reporting',
    'description': 'If you see inaccuracies in the results, please visit: ',
    'links': ['https://www.arin.net/resources/registry/whois/inaccuracy_reporting/']},
   {'title': 'Copyright Notice',
    'description': 'Copyright 1997-2023, American Registry for Internet Numbers, Ltd.',
    'links': None}],
  'links': ['https://rdap.arin.net/registry/ip/23.96.0.0',
   'https://whois.arin.net/rest/net/NET-23-96-0-0-1'],
  'events'

In [59]:
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

def in_order_traversal(root, result):
    if root:
        in_order_traversal(root.left, result)
        result.append(root.val)
        in_order_traversal(root.right, result)

def post_order_traversal(root, result):
    if root:
        post_order_traversal(root.left, result)
        post_order_traversal(root.right, result)
        result.append(root.val)

# Creating the tree
root = TreeNode(2)
root.left = TreeNode(0)
root.left.left = TreeNode(3)
root.left.left.left = TreeNode(9)
root.left.left.right = TreeNode(1)
root.left.right = TreeNode(4)
root.right = TreeNode(8)
root.right.left = TreeNode(6)
root.right.left.left = TreeNode(7)
root.right.right = TreeNode(5)

# Perform in-order traversal
in_order_result = []
in_order_traversal(root, in_order_result)
print("In-order Traversal: ", in_order_result)

# Perform post-order traversal
post_order_result = []
post_order_traversal(root, post_order_result)
print("Post-order Traversal: ", post_order_result)


In-order Traversal:  [9, 3, 1, 0, 4, 2, 7, 6, 8, 5]
Post-order Traversal:  [9, 1, 3, 4, 0, 7, 6, 5, 8, 2]


in-order left mid right
post     left right mid